In [3]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import load_model
import numpy as np
import mimetypes
import argparse
import imutils
import cv2
import os

In [4]:
# define the base path to the input dataset and then use it to derive
# the path to the images directory and annotation CSV file
BASE_PATH = "./input/stanford-dogs-dataset/"
IMAGES_PATH = os.path.sep.join([BASE_PATH, "images/Images/"])
ANNOTS_PATH = os.path.sep.join([BASE_PATH, "annotations/Annotations"])

In [5]:
# define the path to the base output directory
BASE_OUTPUT = "output"
# define the path to the output serialized model, model training plot,
# and testing image filenames
MODEL_PATH = os.path.sep.join([BASE_OUTPUT, "detector.h5"])
PLOT_PATH = os.path.sep.join([BASE_OUTPUT, "plot.png"])
TEST_FILENAMES = BASE_OUTPUT + "/test_images.txt"

In [6]:
# construct the argument parser and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--input", required=True,
# 	help="path to input image/text file of image filenames")
# args = vars(ap.parse_args())

In [7]:
# determine the input file type, but assume that we're working with
# single input image

# filetype = mimetypes.guess_type(args["input"])[0]
# imagePaths = [args["input"]]
# filename = "input/test-images/chihuahua.jpg"
# filetype = filename.split(".")[1]
# # if the file type is a text file, then we need to process *multiple*
# # images
# if "text/plain" == filetype:
# 	# load the filenames in our testing file and initialize our list
# 	# of image paths
# 	# filenames = open(args["input"]).read().strip().split("\n")
# 	filename = filename
# 	imagePaths = []
# 	# loop over the filenames
# 	for f in filenames:
# 		# construct the full path to the image filename and then
# 		# update our image paths list
# 		p = os.path.sep.join([IMAGES_PATH, f])
# 		imagePaths.append(p)

In [17]:
from os import walk
# imagePaths = []
# imagePaths.append("input/test-images/chihuahua.jpg")
# imagePaths.append("input/stanford-dogs-dataset/images/Images/n02086646-Blenheim_spaniel/n02086646_20.jpg")

#Define paths
images_dir = "./input/test-images"
imagePaths = next(walk(images_dir), (None, None, []))[2]  # [] if no file

# load our trained bounding box regressor from disk
print("[INFO] loading object detector...")
model = load_model(MODEL_PATH)
# loop over the images that we'll be testing using our bounding box
# regression model
for imagePath in imagePaths:
	imagePath = images_dir + "/" + imagePath
	# load the input image (in Keras format) from disk and preprocess
	# it, scaling the pixel intensities to the range [0, 1]
	image = load_img(imagePath, target_size=(224, 224))
	image = img_to_array(image) / 255.0
	image = np.expand_dims(image, axis=0)

    # make bounding box predictions on the input image
	preds = model.predict(image)[0]
	(startX, startY, endX, endY) = preds
	# load the input image (in OpenCV format), resize it such that it
	# fits on our screen, and grab its dimensions
	image = cv2.imread(imagePath)
	image = imutils.resize(image, width=600)
	(h, w) = image.shape[:2]
	# scale the predicted bounding box coordinates based on the image
	# dimensions
	startX = int(startX * w)
	startY = int(startY * h)
	endX = int(endX * w)
	endY = int(endY * h)
	# draw the predicted bounding box on the image
	cv2.rectangle(image, (startX, startY), (endX, endY),
		(0, 255, 0), 2)
	# show the output image
	cv2.imshow("Output", image)
	cv2.waitKey(0)

[INFO] loading object detector...
